# Measurement of matter antimatter differences with the LHCb experiment

![](http://lhcb-public.web.cern.ch/lhcb-public/en/LHCb-outreach/multimedia/LHCbDetectorpnglight1.png)

# Introduction
### Hit the grey arrow to extend a section

<b> Welcome to the HEP lab course of the TU Dortmund LHCb group</b>

<div align="justify"> This notebook is going to help you analysing data from the Large Hadron Collider (LHC) at CERN. The goal of your analysis is to make differences visible in the interactions of matter and [antimatter](https://en.wikipedia.org/wiki/Antimatter). This lab course is designed to make you capable to perform data analysis of HEP data on your onw, on  a level similiar to basic researches performed at CERN. Basic understanding of particle physics used to be sufficient and no very sophisticated knowledge is required.
    


# First steps

## Goal:
* Get familiar with the principles of this notebook
* Read simulated data
____

<div align="justify"> Like researchers at CERN you are going to write your own analysis code. To do so you are going to utilize `python`. Though, no experiences in using `python` are expected. There will be a sufficient amount of hints and examples with the exercises. If you need more help using python, there are [tutorials](http://www.tutorialspoint.com/python/) in the internet.

Examples of specific code that could be helpful for this analysis could be found in the example of an [analysis of Nobel laureates](Example-Analysis.ipynb). In this analysis utilizes similiar techniques as you will need for this course. Reading and understanding the code of the nobel laureates analysis could therefore make this lab course significantly easier.

Important: Each of the following code boxes will be executed by hitting Shift+Enter, after selecting the specific box.
The replacement of `In [x]:` (x is empty or a integer) by `In [*]:` next to a code box indicates, that the code is currently running.


## Reading simulated data

Simulated data are a great tool to develop your code and ensure it works correctly before analysing real data from LHCb. The latter are more difficult to analyse, since they contain two types of events. One being the events you are actually interested in, referred to as signal, and the ones that only look or behave similar, called background. In reality, the recorded data are also limited by the resolution of the detector.
The data samples provided for this exersice only contain signal events under the assumption of a perfect detector. 

In [ ]:
from __future__ import print_function
from __future__ import division

%pylab inline
# Setup the notebook

import functools
import os
import warnings
from subprocess import check_output

import numpy
from ipywidgets import interact
import pandas
from root_pandas import read_root
from scipy import stats as st
from matplotlib import pyplot as plt



warnings.filterwarnings("ignore")
rcParams['image.cmap'] = 'Blues'  # change default colormap
pandas.set_option('display.max_columns', None)

If you need help with coding there are, in addition to the [example code](Example-Analysis.ipynb), hints in each section and an overview of useful functions in the instructions. Alternatively you can ask your supervisor for advice.

In [ ]:
# Let us now load the simulated data
sim_data = read_root('/data/PhaseSpaceSimulation.root')

<div align="justify">Now that you can access the data, you can use a number of functions which can help you analyse it. You can find these functions in the libraries imported at the top of the page. Try to make a table of some of the information within your data file so that you can get a feeling of the typical values for attributes in the dataset. Understanding the range of values for different variables will help with plotting graphs.</div>

The data contains information about 'events' that were observed in the detector. An event refers to the particles produced when two proton bunches of the beams collided at the LHC. If you think of the data as a table, each row of the table is the results from a different collision and each column is a different measured quantity. 
 
We are interested in analysing the decays of particles called B<sup>+</sup> or B<sup>-</sup> mesons decaying into three other mesons called kaons (K<sup>+</sup> or K<sup>-</sup>). The events you have been given are those in which this process may have occurred. The detector has been used to reconstruct tracks that may have come from the kaons. You are given the measured momenta, charge, and likelihood of the tracks being kaons. You are given information for three tracks in each event, the ones that could be the three kaons that a B<sup>+</sup> or B<sup>-</sup> meson has decayed into. The instructions give an overview about the recorded information for each event.

In [ ]:
# make a table of the data variables here 
sim_data.describe()

In [ ]:
sim_data.head()

### Hints

**Creating a table** - Use your `head()` - remember to look at the [example analysis](Example-Analysis.ipynb) to see how it was done there.

# Reconstruction of the invariant mass

## Aims:
* Plot histograms of the momenta of one of the kaon candidates
* Calculate the energy of each of the kaon candidates
* Plot the invariant masses of the B<sup>+</sup> or B<sup>-</sup> mesons

### Plotting a feature:

You can plot any features of the data in a histogram. Choose any suitable binning that allows you to observ the distribution of the variable clearly. Try making a histogram for the first kaon candidate's momentum x-component (H1_PX):



In [ ]:
def plot_hist(data, var_name, xlab = "Empty", ylab = "Density", title = "Empty", to_gev = True, range_ = None, n_bins = 50):
    plt.figure(figsize = (12, 7))
    if to_gev:
        var_to_plot = data[var_name]/1000
        plt.hist(x = var_to_plot, bins = n_bins, density = True, range = range_)
    else:
        plt.hist(data = data, x = var_name, bins = n_bins, density = True, range = range_)
    plt.ticklabel_format(style='plain', axis='x', useOffset=False) # Apparently need this for clear tick labels
    if (xlab is not "Empty"):
        plt.xlabel(xlab)
    plt.ylabel(ylab)
    if (title is not "Empty"):
        plt.title(title)
    plt.grid()
    plt.show()

Momentum is a **vector** quantity, it has x,y, and z components. Compare the distributions of the different momentum components.
Try calculating the **magnitude** of the momentum of the first kaon candidate and plot a histogram of the resulting distribution. (You'll need the `H1_PX`, `H1_PY` and `H1_PZ` variables.)



In [ ]:
# make a histogram of the H1_PX, H1_PY and H1_PZ variables here
plot_hist(sim_data, "H1_PX", "First kaon candidate $p_x$ [GeV/c]", title = "Histogram of first kaon candidate $p_x$")
plot_hist(sim_data, "H1_PY", "First kaon candidate $p_y$ [GeV/c]", title = "Histogram of first kaon candidate $p_y$")
plot_hist(sim_data, "H1_PZ", "First kaon candidate $p_z$ [GeV/c]", title = "Histogram of first kaon candidate $p_z$")

In [ ]:
# calculate a variable for the magnitude of the momentum of the first kaon 
sim_data["H1_Pmag"] = np.sqrt(sim_data["H1_PX"]**2 + sim_data["H1_PY"]**2 + sim_data["H1_PZ"]**2)

In [ ]:
sim_data.describe()

### Hints

**Histogram plotting** - You can use the `hist()` function. The parameters `bins(n)` and `range(x,y)` allow you to chose the binning and the range of the histogram.

## Energy and mass

Einstein's theory of special relativity relates energy, mass and momentum.
The momentum of the kaon candidates is measured with the detector (For one candidate you already plotted the components and the magnitude). The invariant mass of the kaon is well known and you can look it up. We wish to determine the energy of the kaons. With all these information you can calculate the energy of the kaon candidates easily using:


<latex> $E^2 = p^2 + m^2$ <latex>

In [ ]:
# calculate the energy of the first kaon
kaon_mass = 493.677 #MeV
sim_data["H1_E"] = np.sqrt(sim_data["H1_Pmag"]**2 + kaon_mass**2)

In [ ]:
# plot a histogram of this variable
plot_hist(sim_data, "H1_E", "First kaon candidate $E$ [GeV]", title = "First kaon candidate $E$")

### Hints

**Energy calculation** - Use the magnitude of momentum variable you calculated above and the known invariant mass of the kaon to work out the energy of the first hadron. Calculate the energy squared, and then the energy and plot this.

**Kaon mass** - you can find the kaon mass on wikipedia or in physics textbooks. A commonly used reference for these kind of information is the Particle Data Group ([PDG](http://pdg.lbl.gov/2014/reviews/rpp2014-rev-charged-kaon-mass.pdf)). All knowledge about the properties of discovered particles are collected there.


Calculate the momenta and energies of the second and third kaon candidates as well.

In [ ]:
# calculate variables for the energy of the other two kaons
sim_data["H2_Pmag"] = np.sqrt(sim_data["H2_PX"]**2 + sim_data["H2_PY"]**2 + sim_data["H2_PZ"]**2)
sim_data["H3_Pmag"] = np.sqrt(sim_data["H3_PX"]**2 + sim_data["H3_PY"]**2 + sim_data["H3_PZ"]**2)

sim_data["H2_E"] = np.sqrt(sim_data["H2_Pmag"]**2 + kaon_mass**2)
sim_data["H3_E"] = np.sqrt(sim_data["H3_Pmag"]**2 + kaon_mass**2)

In [ ]:
plot_hist(sim_data, "H2_E", "Second kaon candidate $E$ [GeV]", title = "Second kaon candidate $E$")
plot_hist(sim_data, "H3_E", "Third kaon candidate $E$ [GeV]", title = "Third kaon candidate $E$")

## Adding features of the $B$ meson

In this analysis we are looking for B<sup>+</sup> or B<sup>-</sup> mesons (see [B meson](https://en.wikipedia.org/wiki/B_meson)) that have decayed into the three charged [kaons](https://en.wikipedia.org/wiki/Kaon).

Energy is a conserved quantities. This means that you can use the energy of the three 'daughter' kaons, which you have calculated above, to calculate the energy that the B meson that decayed into them must have.

Momentum is also a conserved quantity. Hence you can also use the momenta of the 'daughter' kaons to calculate the momentum of the B meson. But be careful - momentum is a **vector** quantity. 

Using the energy and the magnitude of the momentum you can use the energy-momentum relationship to calculate the invariant mass of the B meson.

In [ ]:
# calculate the energy of the B meson
sim_data["B_E"] = sim_data["H1_E"] + sim_data["H2_E"] + sim_data["H3_E"]

In [ ]:
sim_data.describe()

In [ ]:
# calculate the momentum components of the B meson 
sim_data["B_PX"] = sim_data["H1_PX"] + sim_data["H2_PX"] + sim_data["H3_PX"]
sim_data["B_PY"] = sim_data["H1_PY"] + sim_data["H2_PY"] + sim_data["H3_PY"]
sim_data["B_PZ"] = sim_data["H1_PZ"] + sim_data["H2_PZ"] + sim_data["H3_PZ"]
    
# and the magnitude of the momentum of the B meson
sim_data["B_Pmag"] = np.sqrt(sim_data["B_PX"]**2 + sim_data["B_PY"]**2 + sim_data["B_PZ"]**2)

In [ ]:
# calculate the B meson invariant mass
sim_data["B_invM"] = np.sqrt(sim_data["B_E"]**2 - sim_data["B_Pmag"]**2)

# plot the B meson invariant mass in a histogram
plot_hist(sim_data, "B_invM", "m($H_1 H_2 H_3$)[MeV/$c^2$]", title = "Invariant mass of $B^\pm$ meson", to_gev = False, range_ = (5279, 5279.5), n_bins = 100)

In [ ]:
counts, bin_edges = np.histogram(sim_data["B_invM"], bins=100, range = (5279, 5279.5))
max_bin_index = np.argmax(counts)
peak = (bin_edges[max_bin_index] + bin_edges[max_bin_index + 1])/2
print(f"We have a peak at {peak:.2f} MeV.")

You should have a graph that sharply peaks at the mass of the B<sup>+</sup> meson. The mass of the B<sup>+</sup> and B<sup>-</sup> meson are the same. Check that the peak of your graph is at the [known mass](http://pdg.lbl.gov/2014/listings/rpp2014-list-B-plus-minus.pdf) of the B meson. 

Recall that you have made this plot for simulated data. How might you expect the plots for real data to look different ? 

**Congratulations, you finished your first analysis steps!**
In the next section you will start to work with the real LHC data.

### Hints

**B Meson Energy** - From energy conservation, the energy of the B meson will be the sum of the energies of the three kaons: $E_B=E_{K1}+E_{K2}+E_{K3}$, where $E_B$ is the energy of the B meson, $E_{K1}, E_{K2}, E_{K3}$ are the energies of each of the kaons. 

**B meson momentum** - From momentum conservation, the X component of the momentum of the B meson will be the sum of the X momentum components of the three Kaons : $px_B=px_{K1}+px_{K2}+px_{K3}$, where $px$ is the X direction component of the momentum of the B meson, $px_{K1},px_{K2},px_{K3}$ are the X direction components of the momenta of the three kaons. You can then do the same with the Y and Z components. Having obtained the X,Y, and z components of the B momentum you can find the magnitude of the momentum of the B meson.

**B meson invariant mass*** - Rearrange the equation $E^2=p^2+m^2$ to find $m^2$. Using the values of the magnitude of the momentum of the B meson and the B meson Energy, find the mass of the B meson.

**Histogram plotting** - Take care that the range of your mass plot is set suitably that you can see the mass peak.

**Units** - The data you are provided use the unit 'MeV' (10<sup>6</sup> electron volts) for momenta.


# Working with real data
## Aims:

* Filter out data that is not coming from the B<sup>+</sup> → K<sup>+</sup>K<sup>+</sup>K<sup>−</sup> decay channel, or the antiparticle equivalent B<sup>-</sup> → K<sup>+</sup>K<sup>-</sup>K<sup>−</sup>

* Plot a histogram of B-meson mass for the real data and observe how different cuts affect the data

In the previous section simulated data have been used to determine the invariant B<sup>±</sup> mass. Now you are going to utilize real LHCb data, recorded in 2011 during the first major data taking period of the LHC.

The data you are given has been filtered to select only events that are likely to have come from B<sup>+</sup> or B<sup>-</sup> mesons decaying into three final state charged particles. You are interested in the case where these three final state paticles are charged kaons K<sup>+</sup> or K<sup>-</sup>.


## Preselection
You want to apply a preselection to the data to select only events that have a high probability to contain three kaons in the final state. Therefore, you will need to:
* Ensure that they are not muons (use `H1_isMuon` and the equivilant for `H2` and `H3`)
* Require that they each have a low probability of being pions (e.g. `H1_ProbPi < 0.5`)
* Require that they each have a high probability of being a kaon (e.g. `H1_ProbK > 0.5`)

You need to find a balance between making cuts that are too loose and include too many background events and too tight and reject too many of your signal events.
Feel free to come back to this stage later and adjust your cuts to see the impact. 

Now develop your preselection and name it `'preselection'`. We have provided an example preselection in the hints, so feel free to use that to get started if you wish. Redefine the preselection after you have studied the plots and check the impact.

In [ ]:
# Make your preselection here, this line applies no preselection
preselection = "!H1_isMuon & !H2_isMuon & !H3_isMuon & H1_ProbPi < 0.5 & H2_ProbPi < 0.5 & H3_ProbPi < 0.5 & H1_ProbK > 0.5 & H2_ProbK > 0.5 & H3_ProbK > 0.5"

This next line of code just loads the real data into a new DataFrame and applies your created preselection`'preselection'`. This may take a few minutes due to the size of the data sample.

In [ ]:
real_data = read_root(['/data/B2HHH_MagnetUp.root', '/data/B2HHH_MagnetDown.root'], where=preselection)

In [ ]:
real_data.head()

Make histograms of the probability of a final state particle being a kaon or a pion.
These will help to guide you on suitable probability values to cut on.

You can also consider more sophisticated options like 2-D plots of kaon and pion probabilities or different values of the cuts for the different final state particles.

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(real_data["H1_ProbK"], color="red", alpha=1, histtype="step", bins=20, label="First Kaon Candidate", density=True)
plt.hist(real_data["H2_ProbK"], color="green", alpha=1, histtype="step", bins=20, label="Second Kaon Candidate", density=True)
plt.hist(real_data["H3_ProbK"], color="blue", alpha=1, histtype="step", bins=20, label="Third Kaon Candidate", density=True)
plt.title("Probability of a final state particle being a Kaon")
plt.xlabel("Probability")
plt.xlim(0,1)
plt.ylabel("Density")
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend(loc="best")
plt.show()

In [ ]:
# plot the probability that a final state particle is a kaon
all_kaons = np.concatenate([real_data["H1_ProbK"].values, real_data["H2_ProbK"].values, real_data["H3_ProbK"].values])
plot_hist({"all_kaons": all_kaons}, "all_kaons", "Probability", title="Probability of the final state particle being a Kaon", to_gev=False, range_=(0,1), n_bins=50)

In [ ]:
plt.figure(figsize=(9,6))
plt.hist(real_data["H1_ProbPi"], color="red", alpha=1, histtype="step", bins=20, label="First Pion Candidate", density=True)
plt.hist(real_data["H2_ProbPi"], color="green", alpha=1, histtype="step", bins=20, label="Second Pion Candidate", density=True)
plt.hist(real_data["H3_ProbPi"], color="blue", alpha=1, histtype="step", bins=20, label="Third Pion Candidate", density=True)
plt.title("Probability of a final state particle being a Pion")
plt.xlabel("Probability")
plt.xlim(0,1)
plt.ylabel("Density")
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend(loc="best")
plt.show()

In [ ]:
#plot the probability that a final state particle is a pion
all_pions = np.concatenate([real_data["H1_ProbPi"].values, real_data["H2_ProbPi"].values, real_data["H3_ProbPi"].values])
plot_hist({"all_pions":all_pions}, "all_pions", "Probability", title="Probability of the final state particle being a Pion", to_gev=False, range_=(0,1), n_bins=50)

In [ ]:
# 2D plot for first candidates
plt.figure(figsize=(10,6))
plt.hist2d(real_data["H1_ProbK"], real_data["H1_ProbPi"], bins=50)
plt.colorbar(label="Counts")
plt.title("Kaon and Pion Probabilities for First Candidate Particle")
plt.xlabel("Kaon Probability")
plt.ylabel("Pion Probability")

In [ ]:
# 2D plot for second candidates
plt.figure(figsize=(10,6))
plt.hist2d(real_data["H2_ProbK"], real_data["H2_ProbPi"], bins=50)
plt.colorbar(label="Counts")
plt.title("Kaon and Pion Probabilities for Second Candidate Particle")
plt.xlabel("Kaon Probability")
plt.ylabel("Pion Probability")

In [ ]:
# 2D plot for third candidates
plt.figure(figsize=(10,6))
plt.hist2d(real_data["H3_ProbK"], real_data["H3_ProbPi"], bins=50)
plt.colorbar(label="Counts")
plt.title("Kaon and Pion Probabilities for Third Candidate Particle")
plt.xlabel("Kaon Probability")
plt.ylabel("Pion Probability")

In [ ]:
import matplotlib.gridspec as gridspec

fig, axs = plt.subplots(1, 3, figsize=(20, 5))

h1 = axs[0].hist2d(real_data["H1_ProbK"], real_data["H1_ProbPi"], bins=50, cmap="Blues")
axs[0].set_title("H1: Kaon vs Pion Probabilities")
axs[0].set_xlabel("Kaon Probability")
axs[0].set_ylabel("Pion Probability")
fig.colorbar(h1[3], ax=axs[0], label="Counts")
plt.grid(True, linestyle="--", alpha=0.3)

h2 = axs[1].hist2d(real_data["H2_ProbK"], real_data["H2_ProbPi"], bins=50, cmap="Blues")
axs[1].set_title("H2: Kaon vs Pion Probabilites")
axs[1].set_xlabel("Kaon Probability")
axs[1].set_ylabel("Pion Probability")
fig.colorbar(h2[3], ax=axs[1], label="Counts")
plt.grid(True, linestyle="--", alpha=0.3)

h3 = axs[2].hist2d(real_data["H3_ProbK"], real_data["H3_ProbPi"], bins=50, cmap="Blues")
axs[2].set_title("H3: Kaon vs Pion Probabilites")
axs[2].set_xlabel("Kaon Probability")
axs[2].set_ylabel("Pion Probability")
fig.colorbar(h3[3], ax=axs[2], label="Counts")
plt.grid(True, linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()

Now calculate the invariant mass of the B meson for the real data and plot a histogram of this. Which differences do you observe compared to the mass plotted with simulated data? Try to explain the differences. 

In [ ]:
# calculate the magnitude of the momentum
real_data["B_PX"] = real_data["H1_PX"] + real_data["H2_PX"] + real_data["H3_PX"]
real_data["B_PY"] = real_data["H1_PY"] + real_data["H2_PY"] + real_data["H3_PY"]
real_data["B_PZ"] = real_data["H1_PZ"] + real_data["H2_PZ"] + real_data["H3_PZ"]
real_data["B_Pmag"] = np.sqrt(real_data["B_PX"]**2 + real_data["B_PY"]**2 + real_data["B_PZ"]**2)

# calculate the energy 
real_data["H1_Pmag"] = np.sqrt(real_data["H1_PX"]**2 + real_data["H1_PY"]**2 + real_data["H1_PZ"]**2)
real_data["H2_Pmag"] = np.sqrt(real_data["H2_PX"]**2 + real_data["H2_PY"]**2 + real_data["H2_PZ"]**2)
real_data["H3_Pmag"] = np.sqrt(real_data["H3_PX"]**2 + real_data["H3_PY"]**2 + real_data["H3_PZ"]**2)
real_data["H1_E"] = np.sqrt(real_data["H1_Pmag"]**2 + kaon_mass**2)
real_data["H2_E"] = np.sqrt(real_data["H2_Pmag"]**2 + kaon_mass**2)
real_data["H3_E"] = np.sqrt(real_data["H3_Pmag"]**2 + kaon_mass**2)

real_data["B_E"] = real_data["H1_E"] + real_data["H2_E"] + real_data["H3_E"]
real_data["B_invmass"] = np.sqrt(real_data["B_E"]**2 - real_data["B_Pmag"]**2)

real_data.head()

In [ ]:
plot_hist(real_data, "B_invmass", "m($H_1 H_2 H_3$)[MeV/$c^2$]", title="Invariant mass of the $B^\pm$ meson", to_gev=False, n_bins = 100)

Experiment with different cuts and observe the impact of them on the invariant mass plot.
You should select a set of cuts which preserves most of the signal while removing as much as possible of the background at the same time.
Once you have finalised the selection on particle identification, make cuts on the reconstructed particle mass to select the events in the B meson mass peak, removing the background events which lie at lower and higher invariant masses. 

In [ ]:
print(real_data["B_invmass"].min(), real_data["B_invmass"].max())

In [ ]:
# m = 5279.32 MeV
mass_peak = real_data[(real_data["B_invmass"]<5400) & (real_data["B_invmass"]>5200)]
plot_hist(mass_peak, "B_invmass", "m($H_1 H_2 H_3$) [MeV/$c^2$]", title="Invariant mass of the $B^\pm$ meson after cuts", to_gev=False, n_bins = 100)

In [ ]:
counts, bin_edges = np.histogram(mass_peak["B_invmass"], bins=100, range = (5200, 5400))
max_bin_index = np.argmax(counts)
peak = (bin_edges[max_bin_index] + bin_edges[max_bin_index + 1])/2
print(f"We have a peak at {peak:.2f} MeV.")

### Hint: Preselection example

This is an example string, showing the syntax and giving you an example for a possible preselection.

In [ ]:
# preselection = "H1_ProbPi < 0.5 & H2_ProbPi < 0.5 & H3_ProbPi < 0.5 & H1_ProbK > 0.5 & H2_ProbK > 0.5 & H3_ProbK > 0.5 & !H1_isMuon & !H2_isMuon & !H3_isMuon"

# Searching for global matter anti-matter differences

In this section you will start to study matter antimatter differences (CP Violation). The attribute 'global' means that you are looking for differences across all ranges of energy and momentum (the kinematics) of the kaons into which the charged B mesons have decayed. Later you will look at 'local' differences in different regions of the kinematics. 

## Aims:
* Calculate the global CP asymmetry 
* Work out the statistical uncertainty
* Determine, if there is evidence for CP violation in this decay

In order to quantify the matter antimatter asymmetry in this process, we will compare the B<sup>+</sup> meson with its anti-particle B<sup>-</sup>. 
Use the provided charge information (`H1_Charge`, `H2_Charge` and `H3_Charge`) to distinguish B<sup>+</sup> and B<sup>-</sup>.

In [ ]:
# make a variable for the charge of the B mesons
real_data["B_Charge"] = real_data["H1_Charge"] + real_data["H2_Charge"] + real_data["H3_Charge"]
B_plus = real_data[real_data["B_Charge"] == 1]
B_minus = real_data[real_data["B_Charge"] == -1]

real_data.head()

In [ ]:
B_plus.head()

Now count the numbers of events of each of the two types (N<sup>+</sup> and N<sup>-</sup>) and the difference between those.

In [ ]:
# make variables for the numbers of positive and negative B mesons
N_plus = len(B_plus)
N_minus = len(B_minus)
print(N_plus, N_minus, N_plus-N_minus)

Use the following equation to determine the Asmmetry:

<latex>$A = \frac{N^+ - N^-}{N^+ + N^-}$</latex>

In [ ]:
# calculate the value of the asymmetry, by using the formula above, and then print it
A = (N_plus-N_minus)/(N_plus+N_minus)
print(f"The value of asymmetry is {A:.3f}.")

### Hints

**Differentiating between B<sup>+</sup> and B<sup>-</sup>**

 - Charge is a conserved quantity. The charge of the $B$ meson is equal to the sum of the charges of the three daughter particles.
 - You can use ` len(real_data.query('B_Charge == charge'))` to count the number of mesons, where `B_Charge` is the variable you created and `charge` is `1` or `-1`.

### Estimating the significance of the deviation

You will now need to calculate the statistical uncertainty of the asymmetry. You can do so using the formula:

<latex>$\sigma_A = \sqrt{\frac{1-A^2}{N^+ + N^-}}$</latex>

The significance of the result, sigma, is found by dividing the value for asymmetry by its uncertainty. A value exceeding three sigma is considered "evidence" by particle physicists while a value of five sigma or more can be called an "observation" or "discovery".

In [ ]:
# calculate the statistical significance of your result and print it
sigmaA = np.sqrt((1-A**2)/(N_plus+N_minus))
print(f"The statistical uncertainty of the asymmetry is σ = {sigmaA:.3f}.")

In [ ]:
sigma = A/sigmaA
print(f"The significance of the result is {sigma:.3f}.")

**Congratulations!** You have performed your first search for CP violaton.

Till now you have only considered the statistical uncertainty. Your measurement will also have other sources of uncertainty known as systematic uncertainties.
Try to include one systematic uncertainty for the production asymmetry of the $B$ meson and recalculate the significance. 


In [ ]:
sigma_sys = 0.010
sigma_tot = np.sqrt(sigmaA**2 + sigma_sys**2)
sigma_ = A/sigma_tot
print(f"The total uncertainty is {sigma_tot:.3f}.\n")
print(f"The significance of the result is {sigma_:.3f}.")

# Dalitz plots and two body resonances
## Aims:
* Produce Dalitz plots of the simulation and real data sample
* Create ordered and binned dalitz plots to improve the visibility of resonances
* Identify two body resonances in the Dalitz plots

In this stage we introduce you to an important technique for analysing three body decays, in our case the decay of charged B mesons to three kaons. This is known as a Dalitz plot. 

The decay of the B meson can proceed either directly to the three-body final state or via an intermediate particle. For example, B<sup>+</sup> → K<sup>+</sup>K<sup>+</sup>K<sup>−</sup>, could proceed through the decay  B<sup>+</sup> → K<sup>+</sup>R<sup>0</sup>, where R<sup>0</sup> is a neutral particle resonance which can decay R<sup>0</sup> → K<sup>+</sup>K<sup>-</sup>. Dalitz plots can be used to identify these resonances which are visible as bands on the Dalitz plot.

The kinematics of a three-body decay can be fully described using only two variables. The energies and momenta of the three kaons are not independent of each other as they all come from the decay of a B meson and energy and momentum are conserved. The axes of the plots conventionally are the squared invariant masses of two pairs of the decay products. The created 2D plots can display structures in their density distribution.

In our decay B<sup>+</sup> → K<sup>+</sup><sub>1</sub>K<sup>+</sup><sub>2</sub>K<sup>−</sup><sub>3</sub>, the kaons are nummerated to distinguish them. Therefore, we can calculate the invariant mass of three possible combinations that could correspond to intermediate resonances: R<sup>0</sup><sub>1</sub> → K<sup>+</sup><sub>1</sub>K<sup>-</sup><sub>2</sub>, R<sup>++</sup><sub>2</sub> → K<sup>+</sup><sub>1</sub>K<sup>+</sup><sub>3</sub> und R<sup>0</sup><sub>3</sub> → K<sup>-</sup><sub>2</sub>K<sup>+</sup><sub>3</sub>. 

The potential resonance R<sup>++</sup><sub>2</sub> would be a doubly charged resonance. We would not expect to see any resonances corresponding to this as mesons are composed of one quark and one anti-quark and their charges cannot add up to two units.

But the uncharged resonances, R<sup>0</sup><sub>1</sub> and R<sup>0</sup><sub>3</sub>, could be observed. Hence you should compute the invariant mass combinations for these. The square of these masses should be used as the Dalitz variables for a 2D plot. 

We suggest you make these plots first for the simulation data. In the simulation there are no intermediate resonances and your plot should be of uniform density inside the range physically allowed by energy and momentum conservation.
	

In [ ]:
# calculate the invariant masses for each possible hadron pair combination
import pandas as pd
def calculate_invariant_mass(row):
    m1 = row['H1_E']
    m2 = row['H2_E']
    m3 = row['H3_E']
    p1 = np.array([row['H1_PX'], row['H1_PY'], row['H1_PZ']])
    p2 = np.array([row['H2_PX'], row['H2_PY'], row['H2_PZ']])
    p3 = np.array([row['H3_PX'], row['H3_PY'], row['H3_PZ']])
    q1 = row['H1_Charge']
    q2 = row['H2_Charge']
    q3 = row['H3_Charge']

    # Find all pairs with neutral charge and return their invariant masses (always two for three kaons)
    pairs = [
        ((q1 + q2) == 0, (m1 + m2, p1 + p2)),
        ((q1 + q3) == 0, (m1 + m3, p1 + p3)),
        ((q2 + q3) == 0, (m2 + m3, p2 + p3)),
    ]
    neutral_masses = []
    for is_neutral, (E, p) in pairs:
        if is_neutral:
            neutral_masses.append(np.sqrt(E**2 - np.sum(p**2)))
    # Always return two columns (order does not matter)
    return pd.Series(neutral_masses[:2], index=["inv_mass_1", "inv_mass_2"])

# Apply the function to both real and simulated data
invariant_masses_sim = sim_data.apply(calculate_invariant_mass, axis=1)
sim_data = pd.concat([sim_data, invariant_masses_sim], axis=1)

In [ ]:
# plot the invariant mass for one of these combinations 
plot_hist(sim_data, "inv_mass_1", "Invariant mass of first neutral combination [MeV/$c^2$]", title="Invariant mass of first neutral combination", to_gev=False, n_bins=100)

In [ ]:
# make a Dalitz plot with labelled axes for the simulation data using the squared two-body masses
plt.figure(figsize=(12, 8))
plt.hexbin(sim_data['inv_mass_1']**2, sim_data['inv_mass_2']**2, gridsize=50, cmap='Blues', mincnt=1)
plt.colorbar(label='Counts')
plt.xlabel('$(M_{1})^2$ [MeV$^2$/c$^4$]')
plt.ylabel('$(M_{2})^2$ [MeV$^2$/c$^4$]')
plt.title('Dalitz Plot for B Meson Decay (Simulation)')
plt.show()

### Hint

**Calculating the invariant mass** - Use the same technique as you did above for the B meson. But instead of using all three finalstate particles use only two.
**Plotting the Dalitz plot** - You can use the `scatter()` function from the `matplotlib` library or use the example given in the example analysis. Remember to use the square of each two-body mass.

## Adding Dalitz plot for real data
Now draw a Dalitz plot for the real data. Check that your two constructed resonances R<sup>0</sup><sub>2</sub> and R<sup>0</sup><sub>3</sub> have a neutral charge.

In [ ]:
# calculate the invariant masses for each possible hadron pair combination in the real data
invariant_masses_real = real_data.apply(calculate_invariant_mass, axis=1)
# concatenate the invariant masses with the real data
real_data = pd.concat([real_data, invariant_masses_real], axis=1)

In [ ]:
# make a Dalitz plot for the real data (with your preselection cuts applied)
plt.figure(figsize=(12, 8))
plt.hexbin(real_data['inv_mass_1']**2, real_data['inv_mass_2']**2, gridsize=50, cmap='Blues', mincnt=1)
plt.colorbar(label='Counts')
plt.xlabel('$(M_{1})^2$ [MeV$^2$/c$^4$]')
plt.ylabel('$(M_{2})^2$ [MeV$^2$/c$^4$]')
plt.title('Dalitz Plot for B Meson Decay (Real Data)')
plt.show()
# make a Dalitz plot with labelled axes for the simulation data

<div align="justify"> Label the axes of the Dalitz plot. Compare the Dalitz plot of the real data with the one for simulation. What are the most striking differences?
</div>

### Ordering the Dalitz variables
The visibility of the resonances in the Dalitz plot can be improved further. The resonances R<sup>0</sup><sub>1</sub> and R<sup>0</sup><sub>3</sub> are both composed of the same particle types, K<sup>+</sup>K<sup>-</sup>, and hence have the same distributions.
It is useful to impose an ordering which distinguishes the resonances to access more information. Therefore, we sort the two resonances in a combination of kaons with the respectively higher mass R<sup>0</sup><sub>High</sub> and one with the corresponding lower mass R<sup>0</sup><sub>Low</sub>. You can now use the mass of these ordered resonances as your Dalitz plot variables, thus effectively "folding" your Dalitz plot so that one axis always has a higher value than the other. The total energy range is now reduced, while still remaining with the same statistics. This leads to a higher event density and therefore much clearer structures in the Dalitz plots.

In [ ]:
# make a new Dalitz plot with a mass ordering of the axes
real_data["inv_mass_R0_low"] = real_data[["inv_mass_1", "inv_mass_2"]].min(axis=1)
real_data["inv_mass_R0_high"] = real_data[["inv_mass_1", "inv_mass_2"]].max(axis=1)
plt.figure(figsize=(12, 8))
plt.hexbin(real_data['inv_mass_R0_low']**2, real_data['inv_mass_R0_high']**2, gridsize=50, cmap='Blues', mincnt=1)
plt.colorbar(label='Counts')
plt.xlabel('$(M_{R^0_{low}})^2$ [MeV$^2$/c$^4$]')
plt.ylabel('$(M_{R^0_{high}})^2$ [MeV$^2$/c$^4$]')
plt.title('Dalitz Plot for B Meson Decay (Real Data, mass ordering)')
plt.show()

### Hint

**Sorting** - You can find the lower/higher mass by elementwise comparison of the determined masses R<sup>0</sup><sub>1</sub> and R<sup>0</sup><sub>3</sub>. The functions `numpy.min(a,b)` and `numpy.max(a,b)` from the `numpy` library can be helpful. Both functions perform an elementwise comparison between the arrays `a` and `b` and return one array filled by either the individual min/max element. 

### Binned Dalitz plot
In addition, the representation of the Dalitz plots can be improved by using a binning scheme. Therefore, you can use the `hist2d()` function to create a 2D histogram.

In [ ]:
# plot a binned Dalitz Plot
# use colorbar() to make a legend for your plot at the side
plt.figure(figsize=(12, 8))
plt.hist2d(real_data['inv_mass_R0_low']**2, real_data['inv_mass_R0_high']**2, bins=50, cmap='Blues')
plt.colorbar(label='Counts')
plt.xlabel('$(M_{R^0_{low}})^2$ [MeV$^2$/c$^4$]')
plt.ylabel('$(M_{R^0_{high}})^2$ [MeV$^2$/c$^4$]')
plt.title('B Meson Decay Dalitz Plot (Real Data, binned)')
plt.grid(True)
plt.show()

## Two body resonances

You can now use your Dalitz plot to identify the intermediate resonances. These resonances create lines with higher event densities in the Dalitz plots. You can use [PDG page](http://pdg.lbl.gov/2015/tables/contents_tables.html) to identify the observed resonances. The tables give the masses and widths of the particles and their decay modes. You are looking for mesons with the masses corresponding to where you see the bands and that decay into K<sup>+</sup>K<sup>-</sup>.

# Searching for local matter anti-matter differences
## Aims:
* Observe matter antimatter differences (CP violation) in regions of the Dalitz plots of the B<sup>+</sup> and B<sup>-</sup> mesons
* Produce plots to display the CP violation in certain regions

In a section above you searched for global CP violation. You probably did not find a result with very high significance. 

CP violation arises from interference between different decay chains with different intermediate resonances to a common final state. Therefore, the strength and the sign of the CP violation might vary in different kinematic regions. Hence, we can use the same equation that we already used for the global CP violation 

<latex>$A = \frac{N^+ - N^-}{N^+ + N^-}$</latex>

But we will use it in different kinematic regions independently.


## Removing charm resonances 

The analysis performed in this exercise is to study the CP violation in charmless $B$ meson decays to kaons. "Charmless" means that the decay does not proceed through a charm quark. However, the most frequent decay of the B mesons occur through the *b* quark decaying into a *c* quark. The majority of these events can be removed by rejecting the events that are proceeding through a D<sup>0</sup> meson (which contains the charm quark).

In the previous section you were most likely able to already identify a D<sup>0</sup> meson resonance in the Dalitz plots. As a next step you should reject all events in this kinematic region to surpress the charm resonance contribution. You could add this step to your already existing preselection. By simply re-executing the previous analysis steps, you could observe the effects this change has on the Dalitz plots.

In [ ]:
charms_removed_real_data = real_data[
    ((real_data["inv_mass_R0_low"]**2 > 0.36e7) | (real_data["inv_mass_R0_low"]**2 < 0.33e7)) &
    ((real_data["inv_mass_R0_high"]**2 > 1.2e7) | (real_data["inv_mass_R0_high"]**2 < 1.12e7))
]

In [ ]:
# plot a binned Dalitz Plot
# use colorbar() to make a legend for your plot at the side
plt.figure(figsize=(12, 8))
plt.scatter(charms_removed_real_data['inv_mass_R0_low']**2, charms_removed_real_data['inv_mass_R0_high']**2, s = 3)
plt.xlabel('$(M_{R^0_{low}})^2$ [MeV$^2$/c$^4$]')
plt.ylabel('$(M_{R^0_{high}})^2$ [MeV$^2$/c$^4$]')
plt.title('B Meson Decay Dalitz Plot (Real Data, binned)')
plt.grid(True)
plt.show()

## Comparing Dalitz plots

Make separate Dalitz plots for the B<sup>+</sup> and the B<sup>-</sup> decays. Local CP violation will show up as asymmetry between both plots.

In order that the statistical error on the asymmetry in each bin is not overly large, the bins need to contain a reasonable number of entries. Therefore, a coarser binning is needed compared to the Dalitz plots that you used to search for the resonances. A suitable initial bin width should be in the order of a few $\text{GeV}^2/\text{c}^4$.

In [ ]:
# make a Dalitz plot for the B+ events
charms_removed_real_data.head()

In [ ]:
B_plus = charms_removed_real_data[charms_removed_real_data["B_Charge"]==1]
B_plus.head()

In [ ]:
bins = 8

x = np.linspace((B_plus["inv_mass_R0_low"]**2).min(), (B_plus["inv_mass_R0_low"]**2).max(), bins+1)
y = np.linspace((B_plus["inv_mass_R0_high"]**2).min(), (B_plus["inv_mass_R0_high"]**2).max(), bins+1)

plt.figure(figsize=(12,8))
plt.hist2d(B_plus["inv_mass_R0_low"]**2, B_plus["inv_mass_R0_high"]**2, bins=[x,y], cmap="Blues")
#plt.xticks(x, [f"{val/1e7:.1f}" for val in x])  
#plt.yticks(y, [f"{val/1e7:.1f}" for val in y]) 
plt.colorbar(label="Counts")
plt.xlabel("$(M_{1})^2$ [MeV$^2$/c$^4$]")
plt.ylabel("$(M_{2})^2$ [MeV$^2$/c$^4$]")
plt.title("Dalitz Plot for $B^+$ Decay")
plt.show()

#plt.scatter(B_plus["inv_mass_R0_low"]**2, B_plus["inv_mass_R0_high"]**2, s=3)

In [ ]:
# make a Dalitz plot for the B- events
B_minus = charms_removed_real_data[charms_removed_real_data["B_Charge"] == -1]
B_minus.head()

In [ ]:
#bins = 15

#x = np.linspace((B_minus["inv_mass_R0_low"]**2).min(), (B_minus["inv_mass_R0_low"]**2).max(), bins+1)
#y = np.linspace((B_minus["inv_mass_R0_high"]**2).min(), (B_minus["inv_mass_R0_high"]**2).max(), bins+1)

plt.figure(figsize=(12,8))
plt.hist2d(B_minus["inv_mass_R0_low"]**2, B_minus["inv_mass_R0_high"]**2, bins=[x,y], cmap="Blues")
#plt.xticks(x, [f"{val/1e7:.1f}" for val in x])
#plt.yticks(y, [f"{val/1e7:.1f}" for val in y])
plt.colorbar(label="Counts")
plt.xlabel("$(M_{1})^2$ [MeV$^2$/c$^4$]")
plt.ylabel("$(M_{2})^2$ [MeV$^2$/c$^4$]")
plt.title("Dalitz Plot for $B^-$ Decay")
plt.show()

#plt.scatter(B_minus["inv_mass_R0_low"]**2, B_minus["inv_mass_R0_high"]**2, s=3)

In [ ]:
# Make a plot showing the asymmetry between these two Daltz plots
# i.e. calculate the asymmetry between each bin of the B+ and B- Dalitz plots and show the result in another 2D plot

bins = 8

x = np.linspace((charms_removed_real_data["inv_mass_R0_low"]**2).min(),
               (charms_removed_real_data["inv_mass_R0_low"]**2).max(), bins+1)
y = np.linspace((charms_removed_real_data["inv_mass_R0_high"]**2).min(),
               (charms_removed_real_data["inv_mass_R0_high"]**2).max(), bins+1)

hist_b_plus, _, _ = np.histogram2d(B_plus["inv_mass_R0_low"]**2, B_plus["inv_mass_R0_high"]**2, bins=[x,y])
hist_b_minus, _, _ = np.histogram2d(B_minus["inv_mass_R0_low"]**2, B_minus["inv_mass_R0_high"]**2, bins=[x,y])

asymmetry = (hist_b_plus-hist_b_minus)/(hist_b_plus+hist_b_minus)

plt.figure(figsize=(12,8))
plt.imshow(asymmetry.T, origin="lower", interpolation="nearest",
           extent=[x[0], x[-1], y[0], y[-1]],
           cmap="bwr", vmin=-1, vmax=1, aspect="auto")
plt.colorbar(label=r"$A_{CP} = \frac{N(B^+) - N(B^-)}{N(B^+) + N(B^-)}$")
plt.xlabel("$(M_{R^0_{low}})^2$ [MeV$^2$/c$^4$]")
plt.ylabel("$(M_{R^0_{high}})^2$ [MeV$^2$/c$^4$]")
plt.title("Local CP Asymmetry Dalitz Plot")
plt.show()

Observing a large asymmetry in some regions of the plot does not necessarily mean you have observed CP violation. If there are only very few events in that region of the plot the uncertainty on that large asymmetry may be large, too. Hence, the value may still be compatible with zero.

You can calculate the statistical uncertainty on the asymmetry, for each bin of the plot, using the same formulas as you used in the global asymmetry section. You can then make a plot showing the uncertainty on the asymmetry.

By dividing the plot showing the asymmetry by the plot showing the statistical uncertainty you can then obtain the significance of the asymmetry in each bin. Plotting the significance provides a way to select regions with CP violation. 

In [ ]:
# Make a plot showing the uncertainty on the asymmetry 
#Asig = abs(A / Aerr)

# calculate the statistical uncertainty
A = asymmetry
N_plus = hist_b_plus
N_minus = hist_b_minus

sigmaA = np.sqrt((1-A**2)/(N_plus+N_minus))

plt.figure(figsize=(12,8))
plt.imshow(sigmaA.T, origin="lower", interpolation="nearest",
          extent=[x[0], x[-1], y[0], y[-1]], 
           cmap="viridis", aspect="auto")
plt.colorbar(label=r"Statistical Uncertainty $\sigma_{A} = \sqrt{\frac{1-A^2}{N^+ + N^-}}$")
plt.xlabel("$(M_{R^0_{low}})^2$ [MeV$^2$/c$^4$]")
plt.ylabel("$(M_{R^0_{high}})^2$ [MeV$^2$/c$^4$]")
plt.title("Statistical Uncertainty of Local CP Asymmetry Dalitz Plot")
plt.show()

In [ ]:
# Make a plot showing the statistical significance of the asymmetry
sigmaA[sigmaA==0]=100
sigma = np.abs(A/np.sqrt(sigmaA*sigmaA+0.01*0.01))

plt.figure(figsize=(12,8))
plt.imshow(sigma.T, origin="lower", interpolation="nearest",
          extent=[x[0], x[-1], y[0], y[-1]], 
           cmap="viridis", aspect="auto")
plt.colorbar(label=r"Statistical Significance $\sigma = |A / \sigma_A|$")
plt.xlabel("$(M_{R^0_{low}})^2$ [MeV$^2$/c$^4$]")
plt.ylabel("$(M_{R^0_{high}})^2$ [MeV$^2$/c$^4$]")
plt.title("Statistical Significance of Local CP Asymmetry Dalitz Plot")
plt.show()

## Observing CP violation
The previous investigations of the asymmetry and its significance should also now make it possible to select a region in the Dalitz plot that shows signs of significant CP violation of a certain size. This can be identified by a collection of contiguous bins with significant positive (or negative) asymmetry. It may help to vary the binning to select the optimal region.

You can now select the corresponding kinematic region in your data. Then, using a simple 1D histogram, plot the invariant mass distributions for the B<sup>+</sup> and B<sup>-</sup> events,
just as you did it for the global asymmetry. However, this time the events are exclusively in the kinematic region of interest. Now repeat the procedure to determine the CP violation for the selected data, count the respective event rate, calculate the asymmetry, its statistical uncertainty and the resulting significance of the measured CP violation.

In [ ]:
# Make a plot showing the invariant mass of the B+ meson particles
# using events from a region of the Dalitz plot showing sizeable CP asymmetries

M1_min = 0.10e7
M1_max = 0.30e7
M2_min = 0.67e7
M2_max = 1.33e7

selection = charms_removed_real_data[((charms_removed_real_data["inv_mass_R0_low"]**2 >= M1_min) & 
                           (charms_removed_real_data["inv_mass_R0_low"]**2) <= M1_max ) &
                         (charms_removed_real_data["inv_mass_R0_high"]**2 >= M2_min) &
                          (charms_removed_real_data["inv_mass_R0_high"]**2 <= M2_max)
                         ]
B_plus_selection = selection[selection["B_Charge"] == 1]
B_minus_selection = selection[selection["B_Charge"] == -1]

In [ ]:
Np = len(B_plus_selection)
Nm = len(B_minus_selection)
print(Np, Nm)

In [ ]:
asymmetry = (Np-Nm)/(Np+Nm)
print(f"The local CP asymmetry is {asymmetry:.3f}.")

In [ ]:
sigmaA_local = np.sqrt((1-asymmetry**2)/(Np+Nm))
print(f"The statistical uncertainty of the asymmetry is σ = {sigmaA_local:.3f}.")

In [ ]:
signf_A = asymmetry/sigmaA_local
print(signf_A)

In [ ]:
sigma_sys = 0.010
sigmatotal = np.sqrt(sigmaA_local**2 + sigma_sys**2)
signf = asymmetry/sigmatotal
print(f"The total uncertainty is {sigmatotal:.3f}.\n")
print(f"The significance of the result is {signf:.3f}.")

In [ ]:
B_plus_selection.head()

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(B_plus_selection["B_invmass"], color="blue", alpha=0.7, density=False, bins=900)
plt.xlabel(r"m($B^+$) [MeV/$c^2$]")
plt.xlim(5200,5400)
plt.ylabel("Counts")
plt.title(r"Invariant Mass Distribution of $B^+$ events")
plt.grid(True, linestyle="--", alpha=0.3)
plt.show()

In [ ]:
# get the peak
counts, bin_edges = np.histogram(B_plus_selection["B_invmass"], bins=900, range=(5000, 6500))
max_bin_index = np.argmax(counts)
peak = (bin_edges[max_bin_index] + bin_edges[max_bin_index + 1])/2
print(f"We have a peak at {peak:.2f} MeV.")

In [ ]:
# Make a plot showing the invariant mass of the B- meson particles using events from the same region
plt.figure(figsize=(12,6))
plt.hist(B_minus_selection["B_invmass"], color="darkviolet", alpha=0.7, density=False, bins=900)
plt.xlabel(r"m($B^+$) [MeV/$c^2$]")
plt.xlim(5200,5400)
plt.ylabel("Counts")
plt.title("Invariant Mass Distribution of $B^-$ events")
plt.grid(True, linestyle="--", alpha=0.3)
plt.show()

In [ ]:
counts, bin_edges = np.histogram(B_minus_selection["B_invmass"], bins=900, range=(5000, 6500))
max_bin_index = np.argmax(counts)
peak = (bin_edges[max_bin_index] + bin_edges[max_bin_index + 1])/2
print(f"We have a peak at {peak:.2f} MeV.")

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(B_plus_selection["B_invmass"], histtype="step", color="red", alpha=1, density=False, bins=900, label=r"Invariant Mass Distribution of $B^+$ events")
plt.hist(B_minus_selection["B_invmass"], histtype="step", color="blue", alpha=1, density=False, bins=900, label=r"Invariant Mass Distribution of $B^-$ events")
plt.xlabel(r"m($B^\pm$) [MeV/$c^2$]")
plt.xlim(5200,5400)
plt.ylabel("Counts")
plt.title(r"Invariant Mass Distribution of $B^\pm$ events")
plt.grid(True, linestyle="--", alpha=0.3)
plt.legend(loc="best")
plt.show()

**Congratulations!** You should now have succesfully observed significant evidence for CP Violation. Your Dalitz plots should show, that the decay widths of particles and anti-particles differ in different kinematic regions. You may wish to comapre your results with those published by the LHCb collaboration in this [Paper](http://lhcbproject.web.cern.ch/lhcbproject/Publications/LHCbProjectPublic/LHCb-PAPER-2013-027.html).

# Further analyses

The dataset used in your analysis contains an inclusive set of charged B meson decays into three charged tracks recorded at the LHCb experiment. Hence, this dataset has been used for further publicated measurements (you can fin some [here](http://lhcbproject.web.cern.ch/lhcbproject/Publications/LHCbProjectPublic/LHCb-PAPER-2013-027.html) and [here](http://lhcbproject.web.cern.ch/lhcbproject/Publications/LHCbProjectPublic/LHCb-PAPER-2013-051.html)).

There are further possibilities to extend this analysis. The following two possibilities will be allobrated in more details:
<ul>
<li>Additional elements that you could add to your analysis of B<sup>+</sup> → K<sup>+</sup>K<sup>+</sup>K<sup>−</sup> </li>
<li>Similar analyses on this dataset</li>
</ul>

## Adding extra sophistication

### Systematic Uncertainties
In this analysis you considered the statistical uncertainty on the result. This occurs as a result of having only a limited number of events. In addition there are [systematic uncertainties](https://en.wikipedia.org/wiki/Observational_error#Systematic_versus_random_error), these arise from biases in your measurement. Here we discuss three possible sources of those for this analysis.
<ul>
<li> Production asymmetry. The LHC is a proton-proton collider and hence the initial state of the collision is not matter antimatter symmetric. Consequently B<sup>+</sup> and B<sup>-</sup> mesons may not be produced at exactly the same rates. This small production asymmetry it is estimated to be approximately 1%. It can also be measured from the data, as discussed in the LHCb paper.</li>
<li> Detection asymmetry. The LHCb detector could be more efficient for detecting either the B<sup>+</sup> or the B<sup>-</sup> final states. This is because the positive and negative kaons will be bent by the magnet indifferent directions in the detector. If the efficiency of the detector is higher in one region than in another this will lead to higher efficiencies for K<sup>+</sup> or K<sup>-</sup> and hence for B<sup>+</sup> or B<sup>-</sup>. For this reason the magnetic field of the LHCb detector is regularly reversed. You used data in this analysis in which the magnetic field was both up and down and hence the effect will (partially) cancel. By comparing results for the two magnet polarities separately you can check the size of this effect. When loading the data above both polarities were combined, you can instead load them independently to measure the difference between the two datasets.</li>
<li> Analysis technique. The analysis technique you have used may bias the result. A major simplification we made in the analysis above was to neglect 'background' events. We imposed a selection to select a sample of predominantly signal events but have not accounted for the effect of the residual background events.</li>
</ul>

### Using mass sidebands

One source of 'background' events arises from random combinations of tracks in events that happen to fake the 'signal' characteristics. These events will not peak in the mass distribution at the mass of the B meson but will have a rather smoothly varying distribution. Looking at the number and distribution of events away from the mass peak can allow you to estimate the number of background events under the mass peak.

### Fitting distributions

The next level of sophistication in the analysis requires fitting the distributions of events that are observed in the B mass distribution in order to estimate the yield of signal events and background events. You can see how this is done in the LHCb paper on the analysis. Fitting can be performed using for example the [CERN root framework](https://root.cern.ch/), the [zfit](https://zfit.readthedocs.io/en/latest/) or the [iminuit](https://iminuit.readthedocs.io/en/stable/about.html) libaries.

## Further analyses

The LHCb papers using the data set, that you are using, analysed four decay channels of the charged B mesons. You can perform any of these analyses.
<ul>
<li>B<sup>+</sup> → K<sup>+</sup>K<sup>+</sup>K<sup>−</sup> (and anti-particle equivalent). This is the analysis you have performed here. It has the lowest background of the four channels and hence the approximation we made of neglecting the background events will give the least bias to this channel.</li>
<li>B<sup>+</sup> → &pi;<sup>+</sup>&pi;<sup>+</sup>&pi;<sup>−</sup> (and anti-particle equivalent). In this analysis the final state is three charged pions. The level of background events compared to the signal is significantly higher as pions are the most commonly produced particle at the LHC. Hence, a method of estimating the background level should be added to complete this analysis.</li>
<li>B<sup>+</sup> → K<sup>+</sup>&pi;<sup>+</sup>&pi;<sup>−</sup> (and anti-particle equivalent). In this analysis the final state is a mixture of one kaon and two pions. This means that the analysis needs to determine in each event which track is the best candidate kaon and apply selection cuts appropriately to select the events.</li>
<li>B<sup>+</sup> → &pi;<sup>+</sup>K<sup>+</sup>K<sup>−</sup> (and anti-particle equivalent). This channel has a higher level of background compared to the signal.</li>
</ul>